In [1]:
import cv2
import pandas as pd
import numpy as np
from IPython.display import display, Image
import ipywidgets as widgets
from ipywidgets import interact, Layout, GridspecLayout
import cvzone
import threading
import multiprocessing as mp
from multiprocessing.shared_memory import SharedMemory
from importlib import reload
import t1, t2, t3
import pickle

In [2]:
!cp -f df.pkl data/
!cp -f df1.pkl data/
!cp -f l2.pkl data/
!cp -f temp_model_0.pth data/

In [3]:
reload(t1)

width_img = 1280
height_img = 960

with open('data/df.pkl', 'rb') as f:
    df = pickle.load(f)

ID_name = mp.Array('c', b'abcdefghijklmnopqrstuvwxyz')

train = mp.Value('i', 0)
prediction = mp.Value('i', 0)

tmp = mp.Value('i', 0)
tmp2 = mp.Value('i', 0)
contours_size = mp.Value('i', 0)
rectangles_sum = mp.Value('i', 0)

frame_memory = SharedMemory(name='FrameMemory', create=True, size=width_img*height_img*3)
contours_memory = SharedMemory(name='ContoursMemory', create=True, size=4096)
rectangles_memory = SharedMemory(name='RectanglesMemory', create=True, size=100_000)

p1 = mp.Process(target=t2.ocr_detect, args=(tmp, tmp2, width_img, height_img, rectangles_sum, contours_size,), daemon=True)
p1.start()

p2 = mp.Process(target=t3.model_detect, args=(rectangles_sum, prediction, train, tmp2, ID_name), daemon=True)
p2.start()

def b_close(cap):
    cap.release()
    frame_memory.close()
    frame_memory.unlink()
    contours_memory.close()
    contours_memory.unlink()
    rectangles_memory.close()
    rectangles_memory.unlink()
    p1.terminate()
    p2.terminate()

def b_train(ID_widget, ID_name, train, train_button):
    if len(ID_widget.value) > 0:
        ID_name.value = str.encode(ID_widget.value)
        train.value = 1
    train_button.value = False
    
def view(tab, df):
    cap = cv2.VideoCapture(0)
    display_handle=display(None, display_id=True)
    frame_data = np.ndarray((height_img, width_img, 3), dtype=np.uint8, buffer=frame_memory.buf)
    contours_data = np.ndarray((1,4), dtype=np.int32, buffer=contours_memory.buf)
    stack = []
    img_warped = cap.read()[1].copy()

    while True:
        _, frame = cap.read()
        frame_masked = t1.preprocess(frame)
        frame_contours = t1.contours(frame_masked, frame)

        if frame_contours.size != 0:
            img_warped = t1.get_warp(frame, frame_contours)
            contours_size.value = 1
            state = tab.get_state('selected_index')
            if tmp.value == 1:
                frame_data[:,:,:] = img_warped[:,:,:]
                contours_data = np.ndarray((rectangles_sum.value,4), dtype=np.int32, buffer=contours_memory.buf)
                tmp.value = 0
            frame_detect = t1.text_detection(frame, img_warped, contours_data, df, rectangles_sum, train, prediction)
                 
            if state['selected_index'] != 1:
                stack = [frame, img_warped]
            else:
                filt_online = t1.filters_online(img_warped, t1.hue, t1.sat, t1.val, t1.x, t1.y, t1.w, t1.h)
                stack = [frame, filt_online]   
        else:
            # img_warped = frame.copy()
            stack = [frame, frame]
            rectangles_sum.value = 0
            prediction.value = 0
            frame_data[:,:,:] = img_warped[0,0,0]

        img_stack = cvzone.stackImages(stack, 2, 1)

        _, frame = cv2.imencode('.jpeg', img_stack) 
        display_handle.update(Image(data=frame.tobytes()))  

        if train.value == 2:
            with open('data/df.pkl', 'rb') as f:
                df = pickle.load(f)
            train.value = 0
        
        if t1.stop_button.value==True:
            b_close(cap)
            display_handle.update(None)

        if t1.train_button.value==True:
            b_train(t1.ID_widget, ID_name, train, t1.train_button)

display(t1.tab)
thread = threading.Thread(target=view, args=(t1.tab, df,))
thread.start()

None

In [3]:
# Driving Licence (FRONT)
# Driving Licence (BACK)

In [40]:
p1.terminate()
p2.terminate()
frame_memory.close()
frame_memory.unlink()
contours_memory.close()
contours_memory.unlink()
rectangles_memory.close()
rectangles_memory.unlink()

In [41]:
print(p1.is_alive(), p2.is_alive(), thread.is_alive())

False False False


In [3]:
with open('df.pkl', 'rb') as f:
    df = pickle.load(f)
df

,ID,Name
0,0,Karta Polaka (FRONT)
1,1,Karta Polaka (BACK)


In [24]:
with open('df1.pkl', 'rb') as f:
    df1 = pickle.load(f)
df1

,HSV,ID,Watermark_contour
0,[],-1,"[[0, 0, 0, 0]]"
1,"[[0, 179, 160, 255, 0, 255]]",0,"[[5, 5, 220, 295]]"
2,"[[0, 179, 65, 255, 0, 255], [0, 65, 23, 255, 0...",1,"[[320, 5, 670, 500], [1085, 30, 180, 220]]"


In [40]:
df.merge(df1, on='ID')

,ID,Name,HSV,Watermark_contour
0,0,Karta Polaka (FRONT),"[[0, 179, 160, 255, 0, 255]]","[[5, 5, 220, 295]]"
1,1,Karta Polaka (BACK),"[[0, 179, 65, 255, 0, 255], [0, 65, 23, 255, 0...","[[320, 5, 670, 500], [1085, 30, 180, 220]]"
